In [1]:
import json
from pathlib import Path

DST_DIR = Path("./leandojo_benchmark_4")

corpus_path = DST_DIR / 'corpus.jsonl'
lines = list(corpus_path.open())
file_in_corpus = json.loads(lines[2000])
file_in_corpus.keys()

dict_keys(['path', 'imports', 'premises'])

In [2]:
import os

ELAN_PATH = os.path.join(os.environ['HOME'], '.elan', 'bin')
os.environ['PATH'] = ELAN_PATH + ':' + os.environ['PATH']

In [3]:
data_path = DST_DIR / "random/"
split = "test"

from lean_dojo import LeanGitRepo, Theorem, Pos

data = json.load(open(os.path.join(data_path, f"{split}.json")))

theorems = []
positions = []

for t in data:
    repo = LeanGitRepo(t["url"], t["commit"])
    theorems.append(Theorem(repo, t["file_path"], t["full_name"]))
    positions.append(Pos(*t["start"]))

In [4]:
import hashlib

theorems_and_positions = list(zip(theorems, positions))
theorems_and_positions.sort(
        key=lambda x: hashlib.md5(
            f"{x[0].file_path}:{x[0].full_name}".encode()
        ).hexdigest()
    )
theorems, positions = zip(*theorems_and_positions)
theorems, positions = list(theorems), list(positions)

In [5]:
metadata = json.load(open(os.path.join(data_path, "../metadata.json")))
repo = LeanGitRepo(metadata["from_repo"]["url"], metadata["from_repo"]["commit"])

In [6]:
import torch
from typing import Union, List
from transformers import AutoTokenizer, AutoModelForTextEncoding, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-retriever-byt5-small", device_map='cuda')
model = AutoModelForTextEncoding.from_pretrained("kaiyuy/leandojo-lean4-retriever-byt5-small", device_map='cuda')

@torch.no_grad()
def encode(s: Union[str, List[str]]) -> torch.Tensor:
    """Encode texts into feature vectors."""
    if isinstance(s, str):
        s = [s]
        should_squeeze = True
    else:
        should_squeeze = False
    tokenized_s = tokenizer(s, return_tensors="pt", padding=True)
    hidden_state = model(tokenized_s.input_ids.cuda()).last_hidden_state
    lens = tokenized_s.attention_mask.sum(dim=1)
    features = (hidden_state * tokenized_s.attention_mask.unsqueeze(2).cuda()).sum(dim=1) / lens.unsqueeze(1).cuda()
    if should_squeeze:
      features = features.squeeze()
    return features

@torch.no_grad()
def retrieve(state: str, premises: List[str], k: int) -> List[str]:
    """Retrieve the top-k premises given a state."""
    state_emb = encode(state)
    premise_embs = encode(premises)
    scores = (state_emb @ premise_embs.T)
    topk = scores.topk(k).indices.tolist()
    return [premises[i] for i in topk]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
from search_utils import *

corpus = Corpus(DST_DIR / "corpus.jsonl")
corpus_embeddings = torch.zeros(
    len(corpus.all_premises),
    1472,
    dtype=torch.float32,
    device=torch.device("cuda"),
)
corpus_embeddings.shape

2024-06-27 14:59:49.501 | INFO     | search_utils:__init__:210 - Building the corpus from leandojo_benchmark_4/corpus.jsonl


torch.Size([171581, 1472])

When running for the first time, compute the embeddings.

In [8]:
#from tqdm.auto import tqdm

#for i, premise in tqdm(enumerate(corpus.all_premises), total=len(corpus.all_premises)):
#    corpus_embeddings[i] = encode(premise.code).detach()
#torch.save(corpus_embeddings, DST_DIR / "corpus.pt")
#corpus_embeddings.size()

In [9]:
corpus_embedding = torch.load(DST_DIR / "corpus.pt", weights_only=True)

In [10]:
class RAGTacticGenerator:
    def __init__(self, k_premises=16):
        self.retriever = AutoModelForTextEncoding.from_pretrained("kaiyuy/leandojo-lean4-retriever-byt5-small", device_map='cuda')
        self.prover = AutoModelForSeq2SeqLM.from_pretrained("kaiyuy/leandojo-lean4-retriever-tacgen-byt5-small", device_map='cuda')
        self.tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-retriever-tacgen-byt5-small", device_map='cuda')
        self.corpus = Corpus(DST_DIR / "corpus.jsonl")
        self.corpus_embedding = torch.load(DST_DIR / "corpus.pt", weights_only=True)
        self.k_premises = k_premises

    def generate(self, state, file_path, theorem_full_name, theorem_pos, num_samples):
        accessible_premises = self.corpus.get_accessible_premise_indexes(file_path, theorem_pos)
        accessible_premises = torch.topk(
            encode(state) @ self.corpus_embedding[accessible_premises].T,
            self.k_premises
        ).indices
        retrieved_premises = [self.corpus[premise].code for premise in accessible_premises]
        input = "\n\n".join(retrieved_premises + [state])
        tokenized_input = self.tokenizer(input, return_tensors="pt", max_length=2300, truncation=True)
        result = self.prover.generate(
            tokenized_input.input_ids.cuda(),
            max_length=1024,
            num_beams=num_samples,
            length_penalty=0.0,
            do_sample=False,
            num_return_sequences=num_samples,
            early_stopping=False,
            output_scores=True,
            return_dict_in_generate=True,
        )
        tactics = self.tokenizer.batch_decode(result['sequences'], skip_special_tokens=True)
        probs = result[1]
        return [(remove_marks(tac), prob) for tac, prob in zip(tactics, probs)]

In [11]:
prover = BestFirstSearchProver(
    tac_gen=RAGTacticGenerator(),
    timeout=180,
    num_sampled_tactics=16,
    debug=False
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-06-27 15:00:13.474 | INFO     | search_utils:__init__:210 - Building the corpus from leandojo_benchmark_4/corpus.jsonl


In [12]:
solved_count = 0
for i in range(0, 100):
    result = prover.search(theorems[i].repo, theorems[i], positions[i])
    if result is not None and result.status == Status.PROVED:
        solved_count += 1
print(f"Solved {solved_count} out of 100 theorems.")

2024-06-27 15:00:36.288 | INFO     | search_utils:search:378 - Proving Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib4', commit='fe4454af900584467d21f4fd4fe951d29d9332a7'), file_path=PosixPath('Mathlib/Order/Bounds/Basic.lean'), full_name='not_bddBelow_iff')
2024-06-27 15:03:10.433 | INFO     | search_utils:_best_first_search:455 - Found a proof!
2024-06-27 15:03:25.390 | INFO     | search_utils:search:422 - SearchResult(theorem=Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib4', commit='fe4454af900584467d21f4fd4fe951d29d9332a7'), file_path=PosixPath('Mathlib/Order/Bounds/Basic.lean'), full_name='not_bddBelow_iff'), status=<Status.PROVED: 'Proved'>, proof=['simp [BddBelow]', 'simp [Set.Nonempty, lowerBounds]'], actor_time=2.4297626949846745, environment_time=0.769526518881321, total_time=3.223170902580023, num_total_nodes=27, num_searched_nodes=3)
2024-06-27 15:03:25.392 | INFO     | search_utils:search:378 - Proving Theorem(r

Solved 24 out of 100 theorems.


In [13]:
prover = BestFirstSearchProver(
    tac_gen=RAGTacticGenerator(0),
    timeout=180,
    num_sampled_tactics=16,
    debug=False
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-06-27 21:27:21.769 | INFO     | search_utils:__init__:210 - Building the corpus from leandojo_benchmark_4/corpus.jsonl


In [14]:
solved_count = 0
for i in range(0, 100):
    result = prover.search(theorems[i].repo, theorems[i], positions[i])
    if result is not None and result.status == Status.PROVED:
        solved_count += 1
print(f"Solved {solved_count} out of 100 theorems.")

2024-06-27 21:27:43.110 | INFO     | search_utils:search:378 - Proving Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib4', commit='fe4454af900584467d21f4fd4fe951d29d9332a7'), file_path=PosixPath('Mathlib/Order/Bounds/Basic.lean'), full_name='not_bddBelow_iff')
2024-06-27 21:30:25.492 | INFO     | search_utils:_best_first_search:455 - Found a proof!
2024-06-27 21:30:32.739 | INFO     | search_utils:search:422 - SearchResult(theorem=Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib4', commit='fe4454af900584467d21f4fd4fe951d29d9332a7'), file_path=PosixPath('Mathlib/Order/Bounds/Basic.lean'), full_name='not_bddBelow_iff'), status=<Status.PROVED: 'Proved'>, proof=['simp [BddBelow, BddBelow]', 'simp [Set.Nonempty, lowerBounds]'], actor_time=4.457783788442612, environment_time=1.1962122023105621, total_time=5.68161154165864, num_total_nodes=41, num_searched_nodes=5)
2024-06-27 21:30:32.742 | INFO     | search_utils:search:378 - Proving 

AttributeError: 'NoneType' object has no attribute 'status'